<a href="https://colab.research.google.com/github/yinghuing/K_drama_recommender/blob/master/chapter3/exercise1/exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習の準備
---

## 必要なライブラリのインストール

In [ ]:
# !pip install -q langchain langchain-core langchain-openai langchain-community

In [1]:
!pip install -q langchain==0.3.7 langchain-openai==0.2.9 langchain-community==0.3.7 langchain-core==0.3.18 langchain-text-splitters==0.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 587.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 767.8 kB/s eta 0:00:00


## API キーの設定
*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、[**名前**] に `OPENAI_API_KEY` と入力し、その [**値**] に指定されたキーを入力します。
*  [**新しいシークレットを追加**] をクリックし、[**名前**] に `LANGCHAIN_API_KEY` と入力し、その [**値**] に LangSmith で作成してコピーしておいた API キーを入力します。
*  設定した 2 つのシークレットの [**ノートブックからのアクセス**] を有効にします。
*  入力が完了したら、下のセルを実行します。

In [2]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 演習 1.1 会話履歴を保持しないチャットアプリ
---
各タスク (Task:) のコードを実装しながらセルを順次実行してください。

**Task:** Chat model のインスタンスを作成してください
* モデルのタイプ: OpenAI の `gpt-4o-mini`
* temperature: `0`

[langchain_openai.chat_models.base.ChatOpenAI](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [19]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

**Task:** プロンプトのテンプレートを作成してください  
* SystemMessage と HumanMessage からテンプレートを作成します
* SystemMessage: 何か特定の分野についての専門性をもち、ユーザーの質問の回答するように指示してください (分野はお好きなもので構いません)
* HumanMessage: 後でユーザーの入力を代入できるように `{question}` という変数にしてください

[langchain_core.prompts.prompt.PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html)

In [20]:
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a fortune teller and you are specialised in horoscope. Answer the following question:"
    ),
    (
        "human",
        "{question}"
    )
])

**Task:** LLM の回答をテキストの形式に変換する Output parser のインスタンスを作成してください

[langchain_core.output_parsers.string.StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

In [21]:
parser = StrOutputParser()

**Task:** `chat_prompt` > `model` > `parser` の順で処理を実行する Chain を定義してください

In [22]:
chain = chat_prompt | model | parser

次のセルを実行して動作を確認してください

In [23]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = chain.invoke({"question": q})
    print(f'\nAnswer: {answer}')

Write Quit, Exit or Bye to quit.
Your prompt: i am scorpio

Answer: As a Scorpio, you are known for your intensity, passion, and determination. You possess a deep emotional depth and are often seen as mysterious and enigmatic. Your intuition is strong, allowing you to read between the lines and understand the motivations of others.

In the coming weeks, you may find yourself drawn to introspection and self-discovery. This is a great time to explore your inner thoughts and feelings, as the universe encourages you to embrace your transformative nature. Relationships may also take center stage, prompting you to communicate openly and honestly with those you care about.

Remember to harness your natural resilience and focus on your goals. Trust your instincts, and don’t be afraid to dive deep into your passions. The stars are aligned for you to make meaningful connections and pursue your desires with fervor. Embrace the journey ahead, Scorpio!
Your prompt: how is my luck today

Answer: To 

LangSmish でトレース結果を参照し、想定した処理が実行されていたか確認してください。

# 演習 1.2 会話履歴を保持するチャットアプリ
---
下のセル内の各タスク (Task:) のコードを実装してください。  

[langchain_core.runnables.history.RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)  
[How to add memory to chatbots](https://python.langchain.com/docs/how_to/chatbots_memory/)

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Task:
# Chat model のインスタンスを作成
# ・OpenAI の gpt-4o-mini を使用

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# Task:
# 特定のセッション ID に対する会話履歴を取得する関数を作成
# ・ChatMessageHistory を使用

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# Task:
# Prompt template を作成
# ・SystemMessage と HumanMessage からテンプレートを作成します
# ・SystemMessage: 何か特定の分野についての専門性をもち、ユーザーの質問の回答するように指示します (分野はお好きなもので構いません)
# ・HumanMessage: 後でユーザーの入力を代入できるように {input} という変数にします
# ・MessgePlaceholder を使用してプロンプトに会話履歴を含めるようにします
chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are a fortune teller and you are specialised in horoscope. Answer the following question:"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{input}")
    ])



# Task:
# Output parser を作成
# ・LLM からの出力をテキスト形式に変換する Output Parser のインスタンスを作成します

parser = StrOutputParser()


# Task:
# Chain を定義する

chain = chat_prompt | model | parser


# Task:
# RunnableWithMessageHistory を使用して、chain の処理に会話履歴が組み込まれるように実装してください

runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)





次のセルを実行して動作を確認してください

In [25]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = runnable_with_history.invoke({"input": q}, config={"configurable": {"session_id": "b123"}})
    print(f'\nAnswer: {answer}')

Write Quit, Exit or Bye to quit.
Your prompt: i am scorpio

Answer: As a Scorpio, you are known for your intensity, passion, and depth of emotion. You possess a strong intuition and are often seen as mysterious and enigmatic. Your determination and resourcefulness can lead you to achieve great things, but be mindful of your tendency to hold grudges or become overly secretive.

In the coming weeks, the stars suggest that you may find yourself drawn to deeper connections with others. This is a great time to explore your relationships, whether they are romantic or platonic. Embrace vulnerability and open up to those you trust; it could lead to profound insights and stronger bonds.

Additionally, be aware of your emotional landscape. The energy around you may heighten your feelings, so take time for self-reflection and grounding practices. This will help you navigate any challenges that arise with grace and resilience.

Remember, Scorpios thrive in transformation, so embrace any changes th

LangSmish でトレース結果を参照し、想定した処理が実行されていたか確認してください。

お時間に余裕のある方は、プロンプトをいろいろ変えて試してみてください。  
コードを変更していただいても構いません。